Logic Building task: Description: Please create a feature that translates English words into Hindi. The feature should not translate words that start with vowels. If an English word starts with a vowel, the system should display an error message saying, “This word starts with a vowel. Please provide another word.” Additionally, the model should only translate English words that start with vowels between 9 PM and 10 PM. Guidelines: Create your own machine learning with a proper GUI for this task. The GUI should include an input section for entering English words and an output section for displaying the translated Hindi words.

In [ ]:
import numpy as np
import tensorflow as tf
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


eng_sentences = [
    "book", "pen", "dog", "apple", "umbrella",
    "cat", "table", "chair", "car", "bus",
    "train", "plane", "boat", "house", "school",
    "teacher", "student", "bag", "box", "door",
    "window", "wall", "floor", "roof", "fan",
    "light", "clock", "bed", "pillow", "blanket",
    "shirt", "pant", "shoe", "sock", "hat",
    "cap", "glass", "cup", "plate", "spoon",
    "fork", "knife", "bottle", "water", "milk",
    "bread", "rice", "salt", "sugar", "tea",
    "coffee", "juice", "fruit", "banana", "orange",
    "grape", "mango", "lemon", "peach", "pear",
    "plum", "cherry", "berry", "carrot", "potato",
    "onion", "tomato", "garlic", "ginger", "bean",
    "pea", "corn", "wheat", "barley", "oat",
    "egg", "fish", "meat", "chicken", "duck",
    "goat", "sheep", "cow", "buffalo", "horse",
    "donkey", "monkey", "lion", "tiger", "elephant",
    "bear", "wolf", "fox", "deer", "rabbit",
    "mouse", "rat", "bat", "frog", "snake",
    "lizard", "turtle", "crocodile", "whale", "shark",
    "octopus", "crab", "lobster", "bee", "ant",
    "butterfly", "spider", "fly", "mosquito", "worm",
    "flower", "rose", "lotus", "lily", "sunflower",
    "tree", "plant", "leaf", "root", "branch",
    "stem", "seed", "fruit", "bark", "grass",
    "bush", "forest", "jungle", "mountain", "hill",
    "valley", "river", "lake", "pond", "well",
    "sea", "ocean", "island", "desert", "sand",
    "rock", "stone", "pebble", "clay", "mud",
    "dust", "cloud", "rain", "storm", "wind",
    "snow", "ice", "hail", "fog", "dew",
    "sun", "moon", "star", "sky", "space",
    "earth", "world", "land", "field", "farm",
    "village", "city", "town", "market", "shop",
    "store", "mall", "road", "street", "bridge",
    "tunnel", "station", "airport", "port", "harbor",
    "garden", "park", "playground", "hospital", "clinic",
    "doctor", "nurse", "patient", "medicine", "tablet",
    "pill", "syrup", "injection", "bandage", "thermometer",
    "schoolbag", "notebook", "eraser", "sharpener", "scale",
    "color", "paint", "brush", "canvas", "picture",
    "photo", "camera", "film", "video", "computer",
    "laptop", "tablet", "phone", "mobile", "charger",
    "battery", "wire", "plug", "switch", "remote",
    "television", "radio", "music", "song", "dance",
    "movie", "film", "actor", "actress", "director",
    "writer", "poet", "story", "poem", "novel",
    "magazine", "newspaper", "news", "report", "editor",
    "office", "desk", "file", "paper", "envelope",
    "stamp", "letter", "parcel", "post", "mail",
    "bank", "money", "coin", "note", "card",
    "account", "cheque", "cash", "loan", "interest",
    "market", "shopkeeper", "customer", "bill", "receipt",
    "price", "discount", "offer", "sale", "purchase",
    "gift", "present", "toy", "game", "puzzle",
    "ball", "bat", "racket", "net", "goal",
    "team", "player", "coach", "referee", "match",
    "score", "win", "lose", "draw", "cup",
    "medal", "trophy", "prize", "certificate", "degree"
]

hin_sentences = [
    "किताब", "कलम", "कुत्ता", "सेब", "छाता",
    "बिल्ली", "मेज", "कुर्सी", "गाड़ी", "बस",
    "रेलगाड़ी", "हवाई जहाज", "नाव", "घर", "स्कूल",
    "शिक्षक", "छात्र", "बैग", "डिब्बा", "दरवाजा",
    "खिड़की", "दीवार", "फर्श", "छत", "पंखा",
    "लाइट", "घड़ी", "बिस्तर", "तकिया", "कंबल",
    "कमीज", "पैंट", "जूता", "मोजा", "टोपी",
    "कैप", "गिलास", "कप", "थाली", "चम्मच",
    "कांटा", "चाकू", "बोतल", "पानी", "दूध",
    "रोटी", "चावल", "नमक", "चीनी", "चाय",
    "कॉफी", "रस", "फल", "केला", "संतरा",
    "अंगूर", "आम", "नींबू", "आड़ू", "नाशपाती",
    "आलूबुखारा", "चेरी", "बेरी", "गाजर", "आलू",
    "प्याज", "टमाटर", "लहसुन", "अदरक", "फली",
    "मटर", "मक्का", "गेहूं", "जौ", "जई",
    "अंडा", "मछली", "मांस", "मुर्गी", "बतख",
    "बकरी", "भेड़", "गाय", "भैंस", "घोड़ा",
    "गधा", "बंदर", "शेर", "बाघ", "हाथी",
    "भालू", "भेड़िया", "लोमड़ी", "हिरन", "खरगोश",
    "चूहा", "चूहे", "चमगादड़", "मेंढक", "सांप",
    "छिपकली", "कछुआ", "मगरमच्छ", "व्हेल", "शार्क",
    "ऑक्टोपस", "केकड़ा", "लॉबस्टर", "मधुमक्खी", "चींटी",
    "तितली", "मकड़ी", "मक्खी", "मच्छर", "कीड़ा",
    "फूल", "गुलाब", "कमल", "कुमुदिनी", "सूरजमुखी",
    "पेड़", "पौधा", "पत्ता", "जड़", "डाली",
    "तना", "बीज", "फल", "छाल", "घास",
    "झाड़ी", "जंगल", "वन", "पहाड़", "पहाड़ी",
    "घाटी", "नदी", "झील", "तालाब", "कुआँ",
    "समुद्र", "महासागर", "द्वीप", "मरुस्थल", "रेत",
    "चट्टान", "पत्थर", "कंकड़", "मिट्टी", "कीचड़",
    "धूल", "बादल", "बारिश", "तूफान", "हवा",
    "बर्फ", "बर्फ़", "ओले", "कोहरा", "ओस",
    "सूरज", "चाँद", "तारा", "आसमान", "अंतरिक्ष",
    "पृथ्वी", "दुनिया", "भूमि", "खेत", "खेत",
    "गाँव", "शहर", "कस्बा", "बाजार", "दुकान",
    "स्टोर", "मॉल", "सड़क", "गली", "पुल",
    "सुरंग", "स्टेशन", "हवाई अड्डा", "बंदरगाह", "पत्तन",
    "बगीचा", "पार्क", "खेल का मैदान", "अस्पताल", "क्लिनिक",
    "डॉक्टर", "नर्स", "मरीज", "दवा", "गोली",
    "टेबलेट", "सीरप", "इंजेक्शन", "पट्टी", "थर्मामीटर",
    "स्कूल बैग", "कॉपी", "रबर", "शार्पनर", "स्केल",
    "रंग", "पेंट", "ब्रश", "कैनवास", "चित्र",
    "फोटो", "कैमरा", "फिल्म", "वीडियो", "कंप्यूटर",
    "लैपटॉप", "टैबलेट", "फोन", "मोबाइल", "चार्जर",
    "बैटरी", "तार", "प्लग", "स्विच", "रिमोट",
    "टीवी", "रेडियो", "संगीत", "गाना", "नृत्य",
    "फिल्म", "सिनेमा", "अभिनेता", "अभिनेत्री", "निर्देशक",
    "लेखक", "कवि", "कहानी", "कविता", "उपन्यास",
    "पत्रिका", "अखबार", "समाचार", "रिपोर्ट", "संपादक",
    "कार्यालय", "डेस्क", "फाइल", "कागज", "लिफाफा",
    "डाक टिकट", "पत्र", "पार्सल", "डाक", "मेल",
    "बैंक", "पैसे", "सिक्का", "नोट", "कार्ड",
    "खाता", "चेक", "नकद", "ऋण", "ब्याज",
    "बाजार", "दुकानदार", "ग्राहक", "बिल", "रसीद",
    "मूल्य", "छूट", "ऑफर", "बिक्री", "खरीदारी",
    "उपहार", "तोहफा", "खिलौना", "खेल", "पहेली",
    "गेंद", "बल्ला", "रैकेट", "जाल", "गोल",
    "टीम", "खिलाड़ी", "कोच", "रेफरी", "मैच",
    "स्कोर", "जीत", "हार", "ड्रा", "कप",
    "पदक", "ट्रॉफी", "इनाम", "प्रमाण पत्र", "डिग्री"
]



eng_tok = Tokenizer()
hin_tok = Tokenizer()
eng_tok.fit_on_texts(eng_sentences)
hin_tok.fit_on_texts(hin_sentences)

X = eng_tok.texts_to_sequences(eng_sentences)
y = hin_tok.texts_to_sequences(hin_sentences)

eng_max = max(len(seq) for seq in X)
hin_max = max(len(seq) for seq in y)

X = pad_sequences(X, maxlen=eng_max, padding='post')
y = pad_sequences(y, maxlen=hin_max, padding='post')
y = np.expand_dims(y, -1)


model = Sequential([
    Embedding(len(eng_tok.word_index)+1, 128, input_length=eng_max),
    LSTM(128),
    RepeatVector(hin_max),
    LSTM(128, return_sequences=True),
    TimeDistributed(Dense(len(hin_tok.word_index)+1, activation='softmax'))
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(X, y, epochs=300, verbose=0)

print("Model is trained")



hin_index_word = {v: k for k, v in hin_tok.word_index.items()}

def translate_logic(word):
    word = word.strip().lower()
    if not word:
        return "Empty input."

    starts_with_vowel = word[0] in "aeiou"
    current_hour = datetime.now().hour

    if starts_with_vowel and current_hour != 21:
        return "This word starts with a vowel. Please provide another word or try again later."


    seq = eng_tok.texts_to_sequences([word])
    padded = pad_sequences(seq, maxlen=eng_max, padding='post')
    pred = model.predict(padded)[0]
    decoded = [np.argmax(p) for p in pred]
    return ' '.join(hin_index_word.get(i, '') for i in decoded if i != 0).strip()



test_words = ["book", "apple", "umbrella", "pen", "orange", "dog"]

print("Testing the Hindi Translation\n")

for word in test_words:
    print(f"Input: {word}")
    print("Output:", translate_logic(word))
    print("-" * 40)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model is trained
Testing the Hindi Translation

Input: book
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
Output: किताब
----------------------------------------
Input: apple
Output: This word starts with a vowel. Please provide another word or try again later.
----------------------------------------
Input: umbrella
Output: This word starts with a vowel. Please provide another word or try again later.
----------------------------------------
Input: pen
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Output: कलम
----------------------------------------
Input: orange
Output: This word starts with a vowel. Please provide another word or try again later.
----------------------------------------
Input: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Output: कुत्ता
----------------------------------------
